<a href="https://colab.research.google.com/github/Romanchenko-RS/ML/blob/main/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D0%B2_%D0%90%D0%9E%D0%A2_%D0%A0%D0%BE%D0%BC%D0%B0%D0%BD%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%A0_%D0%A1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ноутбук занятия

In [5]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2024-10-31 08:34:42--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv.1’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.01s   

2024-10-31 08:34:42 (123 MB/s) - ‘Constraint_Train.csv.1’ saved [1253562/1253562]



In [6]:
import pandas as pd

df = pd.read_csv('Constraint_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [7]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:04<00:00, 1295.77it/s]


In [9]:
from gensim.models.word2vec import Word2Vec

# Пример использования с правильным параметром
%time model_tweets = Word2Vec(sentences, workers=4, vector_size=300, min_count=3, window=5, epochs=15)

CPU times: user 8.28 s, sys: 86.2 ms, total: 8.36 s
Wall time: 7.97 s


In [10]:
model_tweets.wv.most_similar('france')

[('2015', 0.939629852771759),
 ('bags', 0.9107788801193237),
 ('front', 0.8978564143180847),
 ('floor', 0.8951917886734009),
 ('corpses', 0.8891934752464294),
 ('king', 0.8881967663764954),
 ('road', 0.88685142993927),
 ('flour', 0.8846985101699829),
 ('mountain', 0.882224977016449),
 ('jacksonville', 0.881263792514801)]

In [11]:
model_tweets.init_sims()

<ipython-input-11-f1a8336e4a1e>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_tweets.init_sims()


In [12]:
import numpy as np

In [13]:
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [14]:
features = [get_text_embedding(text) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:02<00:00, 2430.13it/s]


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)

In [17]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
from sklearn.metrics import classification_report

In [19]:
predicted = model.predict(X_test)

In [20]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.90      0.91      0.91      1004
        real       0.92      0.91      0.91      1115

    accuracy                           0.91      2119
   macro avg       0.91      0.91      0.91      2119
weighted avg       0.91      0.91      0.91      2119



In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
vec = CountVectorizer()

In [23]:
bow = vec.fit_transform(df.tweet)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.92      0.92      1035
        real       0.92      0.92      0.92      1084

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



In [26]:
labels = (df.label == 'real').astype(int).to_list()

In [27]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [28]:
max_len

1592

In [29]:
from collections import Counter
fd = Counter([len(tokens) for tokens in token_lists])

In [30]:
fd.most_common(10)

[(20, 178),
 (25, 174),
 (22, 170),
 (18, 170),
 (19, 168),
 (21, 168),
 (16, 163),
 (17, 162),
 (15, 160),
 (23, 156)]

In [31]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [32]:
features = [get_word_embedding(text, 200) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:02<00:00, 2380.30it/s]


In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim

In [35]:
len(features[0][0])

300

In [36]:
len(X_train)

4301

In [37]:
len(X_train)

4301

In [38]:
len(X_train[0])

200

In [39]:
len(X_train[0][0])

300

In [40]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [41]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

<ipython-input-41-02003c1debcb>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  in_data = torch.tensor(X_train).float()


In [42]:
in_data.shape

torch.Size([4301, 200, 300])

In [43]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [44]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [45]:
train_one_epoch(in_data, targets)

100%|██████████| 269/269 [04:56<00:00,  1.10s/it]

tensor(0.6918, grad_fn=<BinaryCrossEntropyBackward0>)


In [46]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [47]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [48]:
result = (output > 0.5) == targets_test

In [49]:
result.sum().item() / len(result)

0.5323265691363851

# Tfidf

In [50]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Загрузка и подготовка данных
df = pd.read_csv('Constraint_Train.csv')

# Разметка данных
X = df['tweet']
y = df['label']

# Преобразование текстовых данных в числовой формат
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Обучение модели
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Оценка модели
y_pred = svm_model.predict(X_test)

# Оценка качества модели
print(classification_report(y_test, y_pred))
print(f'Accuracy: {accuracy_score(y_test, y_pred):.3f}')

              precision    recall  f1-score   support

        fake       0.92      0.94      0.93       596
        real       0.95      0.92      0.94       688

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284

Accuracy: 0.932


# Random Forest

In [51]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Загрузка и подготовка данных
df = pd.read_csv('Constraint_Train.csv')

# Разметка данных
X = df['tweet']
y = df['label']

# Преобразование текстовых данных в числовой формат
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Обучение модели
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Оценка модели
y_pred = rf_model.predict(X_test)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.916
Classification Report:
               precision    recall  f1-score   support

        fake       0.88      0.94      0.91       596
        real       0.95      0.89      0.92       688

    accuracy                           0.92      1284
   macro avg       0.92      0.92      0.92      1284
weighted avg       0.92      0.92      0.92      1284



# PyTorch

Изменим архитектуру нейронной сети

In [52]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 128, num_layers=2, dropout=0.2, batch_first=True)
        self.out = nn.Linear(128, 1)

    def forward(self, x):
        _, (longterm, _) = self.lstm(x)
        prediction = torch.sigmoid(self.out(longterm[-1]))
        return prediction


Доработаем модель

In [53]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Увеличиваем количество эпох
for epoch in range(20):
    train_one_epoch(in_data, targets, batch_size=64)

100%|██████████| 68/68 [05:39<00:00,  4.99s/it]


tensor(0.6918, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 68/68 [05:52<00:00,  5.19s/it]


tensor(0.6920, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 68/68 [06:07<00:00,  5.40s/it]


...

tensor(0.6913, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 68/68 [06:29<00:00,  5.73s/it]


tensor(0.6913, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 68/68 [06:32<00:00,  5.77s/it]

tensor(0.6913, grad_fn=<BinaryCrossEntropyBackward0>)


In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Преобразование данных для PyTorch
in_data = torch.tensor(X_train.toarray()).float() if hasattr(X_train, 'toarray') else torch.tensor(X_train).float()
targets = torch.tensor(y_train_encoded).float()

in_data_test = torch.tensor(X_test.toarray()).float() if hasattr(X_test, 'toarray') else torch.tensor(X_test).float()
targets_test = torch.tensor(y_test_encoded).float()

net = Net()

# Оптимизатор и функция потерь
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Функция для обучения одной эпохи
def train_one_epoch(in_data, targets, batch_size=64):
    net.train()
    for i in range(0, in_data.shape[0], batch_size):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(f"Loss: {loss.item()}")

# Обучение модели
for epoch in range(20):
    print(f"Epoch {epoch+1}/20")
    train_one_epoch(in_data, targets, batch_size=64)

Epoch 1/20
Loss: 0.43030688166618347
Epoch 2/20
Loss: 0.1433841735124588
Epoch 3/20
Loss: 0.06065298616886139
Epoch 4/20
Loss: 0.03354205936193466
Epoch 5/20
Loss: 0.021166713908314705
Epoch 6/20
Loss: 0.014382502064108849
Epoch 7/20
Loss: 0.010248785838484764
Epoch 8/20
Loss: 0.007540313992649317
Epoch 9/20
Loss: 0.005719645880162716
Epoch 10/20
Loss: 0.004456173628568649
Epoch 11/20
Loss: 0.0035297521390020847
Epoch 12/20
Loss: 0.0028617912903428078
Epoch 13/20
Loss: 0.0023682720493525267
Epoch 14/20
Loss: 0.001985131995752454
Epoch 15/20
Loss: 0.0016861920012161136
Epoch 16/20
Loss: 0.0014476986834779382
Epoch 17/20
Loss: 0.001254820148460567
Epoch 18/20
Loss: 0.0010978949721902609
Epoch 19/20
Loss: 0.0009659319184720516
Epoch 20/20
Loss: 0.0008554793894290924


In [66]:
# Оценка модели
net.eval()
with torch.no_grad():
    predictions = net(in_data_test).reshape(-1)
    predictions = torch.round(predictions).numpy()

# Расчет метрик
accuracy = accuracy_score(targets_test, predictions)
f1 = f1_score(targets_test, predictions)

print(f"Accuracy: {accuracy:.3f}")
print(f"F1 Score: {f1:.3f}")

Accuracy: 0.926
F1 Score: 0.930
